<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter07/Detectron2_Chapter07_Pixel_Means_and_STD.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 07 - Pixel Means and Standard Deviations
## The dataset
Should execute the Notebook titled "Data Processing" first to process the dataset or run the following code to download the processed dataset from GitHub repository of the book.

In [ ]:
!wget -q https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/datasets/braintumors_coco.zip
!unzip -q braintumors_coco.zip

replace braintumors_coco/train/97 (2).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Preparing a data loader

In [ ]:
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import os
import cv2

# Some configurations
name_ds = "braintumors_coco"
af = "_annotations.coco.json"
img_dir = name_ds + "/train/"
json_file_train = name_ds + "/train/" + af
batch_size = 64
num_workers = 2

In [ ]:
class TumorDataset(Dataset):
  def __init__(self, 
                data,
                img_dir="",
                transform = None):
    self.data = data
    self.img_dir = img_dir
    self.transform = transform
      
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, idx):
    file_name  = os.path.join(self.img_dir, 
                              self.data[idx]['file_name'])
    image = cv2.imread(file_name)
    if self.transform is not None:
        image = self.transform(image = image)['image']
    
    return image

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

image_size = 640
augs = A.Compose([
    A.Resize(height = image_size, width = image_size),
    ToTensorV2()])

In [ ]:
import json
with open(json_file_train) as f:
  ds = json.load(f)['images']

tds = TumorDataset(ds, img_dir=img_dir, transform=augs)
image_loader = DataLoader(tds, 
                          batch_size  = batch_size, 
                          shuffle     = False, 
                          num_workers = num_workers,
                          pin_memory  = True)

# Calculate running means and standard deviations

In [ ]:
def broad_cast(x, image_size, channel):
  y = torch.broadcast_to(x, (image_size**2, channel))
  z = y.reshape(image_size, image_size, 3)
  return torch.moveaxis(z, 2, 0)

In [ ]:
class RunningStats:
  def __init__(self):
    self.n = 0
    self.mean = 0
    self.ssd = 0 
    
  def push(self, x):
    # start
    dims = [0, 2, 3]
    count = 1
    for dim in dims:
      count *= x.shape[dim]
    image_size = x.shape[-1]
    channel = x.shape[1]
    if self.n == 0:
      # start
      new_mean = x.sum(axis=dims)/count
      new_ssd = ((
          x - broad_cast(
                new_mean, 
                image_size, 
                channel
                ))**2).sum(axis=dims)
      new_count = count
    else:
      # old
      old_count = self.n
      old_mean = self.mean
      old_ssd = self.ssd
      old_sum = old_mean * old_count      
      # new
      new_count = self.n + count
      new_sum = old_sum + x.sum(axis=dims)
      new_mean = new_sum/(self.n + count)
      
      old_ssd_new_mean = (
          old_ssd  
          + 2*old_mean*old_sum
          - old_count*(old_mean)**2
          - 2*new_mean*old_sum
          + old_count*(new_mean)**2
          )
      
      new_ssd = (
          old_ssd_new_mean + 
          (
              (x - broad_cast(new_mean, 
                           image_size, 
                           channel))**2
           ).sum(axis=dims))
    # release results
    self.mean = new_mean
    self.ssd = new_ssd
    self.n = new_count
    self.std = torch.sqrt(new_ssd/(new_count-1))

In [ ]:
rs = RunningStats()
for inputs in tqdm(image_loader):
  rs.push(inputs)

print()
print(rs.mean)
print(rs.std)

100%|██████████| 14/14 [00:32<00:00,  2.31s/it]


tensor([20.1962, 20.1962, 20.1962])
tensor([39.5985, 39.5985, 39.5985])
